In [3]:
import pandas as pd
import datetime

import seaborn as sns
sns.set(font_scale=1.3)

import catboost as cb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold


import numpy as np
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, classification_report, confusion_matrix

import warnings

warnings.filterwarnings("ignore")

In [4]:
SEED = 1

In [6]:
train_df_target = pd.read_csv('alfabattle2_abattle_train_target.csv')
submit_df= pd.read_csv('alfabattle2_prediction_session_timestamp.csv')

submit_df_test['timestamp'] = pd.to_datetime(submit_df['timestamp'])
train_df_target['timestamp'] = pd.to_datetime(train_df_target['timestamp'])

In [7]:
submit_df_test['timestamp'] = pd.to_datetime(submit_df['timestamp'])

In [9]:
submit_df.shape

(79268, 2)

## SMALL EDA

In [4]:
train_df_target.multi_class_target.unique()

array(['main_screen', 'invest', 'statement', 'phone_money_transfer',
       'own_transfer', 'credit_info', 'chat', 'card2card_transfer',
       'mobile_recharge', 'card_recharge'], dtype=object)

In [5]:
train_df_target.shape

(5065350, 4)

In [6]:
train_df_target.multi_class_target.value_counts()/train_df_target.multi_class_target.shape[0]

main_screen             0.450268
statement               0.182133
credit_info             0.098453
own_transfer            0.057267
mobile_recharge         0.052609
phone_money_transfer    0.045981
card2card_transfer      0.038177
chat                    0.036478
card_recharge           0.027366
invest                  0.011268
Name: multi_class_target, dtype: float64

In [7]:
train_df_target.head(n=1)

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen


In [15]:
submit_df.client_pin.unique().shape

(79268,)

In [16]:
train_df_target.client_pin.unique().shape

(79268,)

In [17]:
set(train_df_target.client_pin.unique()).difference(submit_df.client_pin.unique())

set()

In [10]:
pd.options.display.max_rows = 100

In [19]:
train_df_target['client_pin'].value_counts().head(n=10)

674595ee5acb8c2ede30dadfa722ba9d    1646
25074c2bd77b64c40a4c6a623a748724    1626
e17a88a65bca752fbb13c2418ba09d18    1550
bdfb2a491cb1a146358319b6955d7308    1535
bb0a6c5a08ae8581854386e8421a3773    1487
9d2d868c1a25534190017c32c4c94299    1452
28a986fe76100867d16a1841630c0a84    1391
7c644b12a138ab77c8932d8e35c3bd9f    1380
d0209cfcb3ff095acd3c97c08c6c46a3    1300
16a958d3c1d280ecac0d93411a9af478    1287
Name: client_pin, dtype: int64

In [20]:
test_user_df = train_df_target[train_df_target['client_pin'] == 'b95c4712fffcef8247a99d2b78af9f8b']

In [21]:
test_user_df.sort_values(by='timestamp', inplace=True)

In [22]:
test_user_df['prev_timestamp'] = test_user_df.groupby('client_pin').timestamp.shift(1)
test_user_df['prev_target'] = test_user_df.groupby('client_pin').multi_class_target.shift(1)

In [32]:
test_user_df['diff'] = test_user_df['timestamp'] - test_user_df['prev_timestamp']

In [33]:
test_user_df['diff'] = test_user_df['diff']/np.timedelta64(1, 'm')

In [35]:
test_user_df[test_user_df['diff'] < 100].head(n=10)

,session_id,client_pin,timestamp,multi_class_target,prev_timestamp,prev_target,diff,day,weekday
2997084,ebc5c56a9df740c1e6088fc253652aaf,b95c4712fffcef8247a99d2b78af9f8b,2020-01-07 13:06:16,main_screen,2020-01-07 11:58:05,main_screen,68.183333,7,1
3305984,31e0e1d3fcacf8f294dcc7f01b550dc5,b95c4712fffcef8247a99d2b78af9f8b,2020-01-10 07:58:17,main_screen,2020-01-10 06:27:32,main_screen,90.750000,10,4
4847477,40f8b674cfebd33c952cd1ca25ee2f5f,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 11:25:59,main_screen,2020-01-17 10:49:47,main_screen,36.200000,17,4
3891100,0c1c809f2ab91b93bbe215382f022160,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 11:42:30,main_screen,2020-01-17 11:25:59,main_screen,16.516667,17,4
1214481,ec864b02c68cda2cc5c364d454993022,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 13:04:58,main_screen,2020-01-17 11:42:30,main_screen,82.466667,17,4
703665,b4491666148e1c80fd12d1ccfd7371eb,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 13:29:42,main_screen,2020-01-17 13:04:58,main_screen,24.733333,17,4
3832489,2580155009df0b857cd27e90e089a95c,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 18:07:58,main_screen,2020-01-17 16:52:57,main_screen,75.016667,17,4
2362031,9183cc1f706f758db63923199c501c24,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 20:11:03,main_screen,2020-01-17 19:52:13,main_screen,18.833333,17,4
3597523,8a3864e0f7cfa05c72079ba977730331,b95c4712fffcef8247a99d2b78af9f8b,2020-01-17 21:10:12,main_screen,2020-01-17 20:11:03,main_screen,59.150000,17,4
486339,04fa2c56456eae47ed932ff884c445a3,b95c4712fffcef8247a99d2b78af9f8b,2020-01-18 11:56:21,main_screen,2020-01-18 10:37:55,main_screen,78.433333,18,5


In [36]:
test_user_df['day'] = test_user_df['timestamp'].apply(lambda x: x.day)
test_user_df['weekday'] = test_user_df['timestamp'].apply(lambda x: x.weekday())

In [38]:
test_user_df[['timestamp','prev_timestamp','diff', 'prev_target', 'multi_class_target', 'day', 'weekday']].head(n=10)

,timestamp,prev_timestamp,diff,prev_target,multi_class_target,day,weekday
4946629,2020-01-01 04:00:39,NaT,NaN,NaN,main_screen,1,2
1320507,2020-01-01 10:33:05,2020-01-01 04:00:39,392.433333,main_screen,main_screen,1,2
1590436,2020-01-01 12:52:17,2020-01-01 10:33:05,139.200000,main_screen,main_screen,1,2
309489,2020-01-01 15:37:08,2020-01-01 12:52:17,164.850000,main_screen,main_screen,1,2
1399544,2020-01-01 17:53:25,2020-01-01 15:37:08,136.283333,main_screen,main_screen,1,2
2742529,2020-01-02 11:27:30,2020-01-01 17:53:25,1054.083333,main_screen,invest,2,3
185478,2020-01-02 19:13:55,2020-01-02 11:27:30,466.416667,invest,main_screen,2,3
4787639,2020-01-03 09:10:58,2020-01-02 19:13:55,837.050000,main_screen,main_screen,3,4
3512952,2020-01-03 14:23:44,2020-01-03 09:10:58,312.766667,main_screen,main_screen,3,4
4606258,2020-01-03 17:18:04,2020-01-03 14:23:44,174.333333,main_screen,main_screen,3,4


### Проверка, что таймстэмпы в сабмите выше трейна

In [4]:
res = train_df_target[['client_pin','timestamp']].merge(submit_df[['client_pin','timestamp']], on='client_pin')

In [40]:
np.mean([res['timestamp_x']> res['timestamp_y']])

0.0

## FEATURE ENGENEERING

In [5]:
df = pd.concat([train_df_target, submit_df])

In [6]:
train_df_target.shape, submit_df.shape, df.shape

((5065350, 4), (79268, 2), (5144618, 4))

In [7]:
df.head(n=1)

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen


### Работа со временем

In [8]:
def process_time(df):
    df['timestamp'] = pd.to_datetime(df.timestamp)
    df['weekday'] = df['timestamp'].apply(lambda x: x.weekday())
    df['day'] = df['timestamp'].apply(lambda x: x.day)
    df['month'] = df['timestamp'].apply(lambda x: x.month)
    df['hour'] = df['timestamp'].apply(lambda x: x.hour)
#     df['minute'] = df['timestamp'].apply(lambda x: x.minute) кажется, что бесполезные
#     df['second'] = df['timestamp'].apply(lambda x: x.second)
    
    return df

In [9]:
df = process_time(df)

In [10]:
df.sort_values(by='timestamp', inplace=True)

### Получение предыдущих таргетов и разниц тайстэмпов между этими таргетами. Подсчет среднего времени между сессиями абонента

In [11]:
deep_target_hist = 11

In [12]:
df['timestamp_0'] = df['timestamp']

In [13]:
from itertools import combinations

In [14]:
for i in range(1, deep_target_hist):
    df[f'target_{i}'] = df.groupby('client_pin').multi_class_target.shift(i)
    df[f'timestamp_{i}'] = df.groupby('client_pin').timestamp.shift(i)
    df.loc[df[f'target_{i}'].isnull(), f'target_{i}'] = 'nan'
    
    time_comb = list(combinations(range(i+1), 2))
    
    for comb in time_comb:
        t_2, t_1 = comb
        df[f'time_diff_{t_1}_{t_2}'] = (df[f'timestamp_{t_2}'] - df[f'timestamp_{t_1}'])/np.timedelta64(1, 'm')

In [15]:
%%time

df['mean_diff_time_global'] = df.groupby('client_pin')['time_diff_1_0'].cumsum()/(df.groupby('client_pin').cumcount())
df['mean_diff_time_global'] = df.groupby('client_pin')['mean_diff_time_global'].shift(1)

df['mean_diff_time_global_explotation'] = df['time_diff_1_0'] / df['mean_diff_time_global']

CPU times: user 7.4 s, sys: 5.09 s, total: 12.5 s
Wall time: 12.5 s


In [16]:
test_user = df[df['client_pin'] == '5b59bf5bd1e07775b1bc95dd7151d2cd']

In [17]:
test_user[['time_diff_1_0', 'mean_diff_time_global', 'timestamp']].head(n=10)

,time_diff_1_0,mean_diff_time_global,timestamp
2332378,NaN,NaN,2020-01-06 15:57:33
3973900,3925.566667,NaN,2020-01-09 09:23:07
4067820,543.350000,3925.566667,2020-01-09 18:26:28
1083722,1535.316667,2234.458333,2020-01-10 20:01:47
1079056,7194.283333,2001.411111,2020-01-15 19:56:04
3953524,23975.183333,3299.629167,2020-02-01 11:31:15
4487205,10104.800000,7434.740000,2020-02-08 11:56:03
2145116,42.266667,7879.750000,2020-02-08 12:38:19
202748,3162.833333,6760.109524,2020-02-10 17:21:09
1681641,22786.233333,6310.450000,2020-02-26 13:07:23


### Количество сессий на абонента

In [18]:
df.sort_values(by='timestamp', inplace=True)

In [19]:
df['sessions_number'] = df.groupby('client_pin').cumcount()

In [2100]:
test_user = df[df['client_pin'] == '586fe605f61e81581efe72d9d3dc237d']

In [2101]:
test_user[['sessions_number','timestamp']].head(n=10)

,sessions_number,timestamp
514708,0,2020-01-01 00:00:01
4442807,1,2020-01-09 21:32:15
2925094,2,2020-01-20 08:57:05
1149497,3,2020-01-28 00:17:55
1037144,4,2020-02-03 16:00:01
2605473,5,2020-02-05 14:54:06
2542136,6,2020-02-19 16:36:55
2201877,7,2020-03-04 18:24:24
1872344,8,2020-03-05 07:51:26
2234217,9,2020-03-19 21:57:19


### Суммарное количество каждого таргета в прошлом

In [22]:
df.loc[df['multi_class_target'].isnull(), 'multi_class_target'] = 'nan'

In [23]:
# df.drop(columns = cols, inplace=True)

In [24]:
buf = pd.get_dummies(df['multi_class_target'], prefix='prev_sum')

buf.drop(columns='prev_sum_nan', inplace=True)

cols = buf.columns

df = pd.concat([buf, df], axis=1)

test_user = df[df['client_pin'] == '5b59bf5bd1e07775b1bc95dd7151d2cd']

for col in cols:
    print(col, buf[col].sum())
    df[col] = df.groupby('client_pin')[col].cumsum()
    df[col] = df.groupby('client_pin')[col].shift(1)

prev_sum_card2card_transfer 193378
prev_sum_card_recharge 138616
prev_sum_chat 184775
prev_sum_credit_info 498698
prev_sum_invest 57078
prev_sum_main_screen 2280763
prev_sum_mobile_recharge 266485
prev_sum_own_transfer 290077
prev_sum_phone_money_transfer 232911
prev_sum_statement 922569


In [29]:
cols

Index(['prev_sum_card2card_transfer', 'prev_sum_card_recharge',
       'prev_sum_chat', 'prev_sum_credit_info', 'prev_sum_invest',
       'prev_sum_main_screen', 'prev_sum_mobile_recharge',
       'prev_sum_own_transfer', 'prev_sum_phone_money_transfer',
       'prev_sum_statement'],
      dtype='object')

In [30]:
df[df['client_pin'] == '5b59bf5bd1e07775b1bc95dd7151d2cd'][cols].head(n=10)

,prev_sum_card2card_transfer,prev_sum_card_recharge,prev_sum_chat,prev_sum_credit_info,prev_sum_invest,prev_sum_main_screen,prev_sum_mobile_recharge,prev_sum_own_transfer,prev_sum_phone_money_transfer,prev_sum_statement
2332378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3973900,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4067820,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1083722,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1079056,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3953524,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4487205,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
2145116,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
202748,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1681641,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


### Количество подряд идущих знаний прошлого таргета, считается долго

In [31]:
lb = LabelEncoder()

In [32]:
df['label'] = lb.fit_transform(df['multi_class_target'])

In [33]:
## Выполняется долго
%%time

def get_seq_last_target_len(df):
    df = df.iloc[:-1]
    
    if len(df) == 0:
        return 0
    
    value = df.iloc[-1]
        
    n_sum = 0
    
    for el in df.iloc[::-1]:
        if el != value:
            break
        else:
            n_sum += 1
    
    return n_sum

df['seq_last_target_len'] = df.groupby('client_pin')['label'].transform(lambda x: x.rolling(window=x.shape[0], min_periods=1).apply(lambda x: get_seq_last_target_len(x)))

### Получения самого частого таргета в прошлом, помогает получение только последнего. Генерация фичей, показывающие отношения между этими таргетами

In [34]:
target_cols = [val for val in df['multi_class_target'].unique() if str(val) !='nan']

In [36]:
%%time
buf = df[cols].rename({col: col.split('prev_sum_')[1] for col in cols}, axis=1)

bigger_cols = []

for pair in list(combinations(target_cols, 2)):
    el_1, el_2 = pair
    
    buf[f'bigger_{el_1}_{el_2}'] = buf[el_1] > buf[el_2]
    bigger_cols.append(f'bigger_{el_1}_{el_2}')
    
df = pd.concat([buf[bigger_cols], df], axis=1)

CPU times: user 5.39 s, sys: 3.72 s, total: 9.11 s
Wall time: 9.13 s


In [37]:
## проверка, что нету дублей
df.columns == 'bigger_mobile_recharge_statement'

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [38]:
df['top_target'] = cols[df[cols].values.argsort(1)[:, -1]]
df['top_target_2'] = cols[df[cols].values.argsort(1)[:, -2]]
df['top_target_3'] = cols[df[cols].values.argsort(1)[:, -3]]

Теперь нужно проставить наны там, где топ предыдущих таргетов на самом деле ещё нету

In [39]:
df.loc[df['sessions_number'] == 0, ['top_target', 'top_target_2', 'top_target_3']] = 'nan'

In [40]:
els = df['top_target_2'].unique()
els = np.delete(els, 0)

In [41]:
%%time

for el in els:
    df.loc[(df['top_target_2'] == el) & (df[el] == 0), ['top_target_2', 'top_target_3']] = 'nan'
    df.loc[(df['top_target_3'] == el) & (df[el] == 0), 'top_target_3'] = 'nan'

CPU times: user 5.52 s, sys: 1.63 s, total: 7.15 s
Wall time: 7.17 s


In [42]:
test_user = df[df['client_pin'] == '5b59bf5bd1e07775b1bc95dd7151d2cd']

In [43]:
test_user[cols.values.tolist() + ['top_target', 'top_target_2', 'top_target_3','timestamp']].head(n=20)

,prev_sum_card2card_transfer,prev_sum_card_recharge,prev_sum_chat,prev_sum_credit_info,prev_sum_invest,prev_sum_main_screen,prev_sum_mobile_recharge,prev_sum_own_transfer,prev_sum_phone_money_transfer,prev_sum_statement,top_target,top_target_2,top_target_3,timestamp
2332378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,2020-01-06 15:57:33
3973900,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-01-09 09:23:07
4067820,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-01-09 18:26:28
1083722,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-01-10 20:01:47
1079056,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-01-15 19:56:04
3953524,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-02-01 11:31:15
4487205,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-02-08 11:56:03
2145116,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-02-08 12:38:19
202748,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-02-10 17:21:09
1681641,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,prev_sum_main_screen,nan,nan,2020-02-26 13:07:23


### Сохранение дф

In [44]:
df.to_pickle('/data/edmitrie/alfa_boosters/train_submit_df_2020_01_01.pickle')

In [45]:
df.shape

(5144618, 147)

## Моделирование

In [3]:
df = pd.read_pickle('/data/edmitrie/alfa_boosters/train_submit_df_2020_01_01.pickle')

In [4]:
res_parquet_subm = pd.read_pickle('/data/edmitrie/alfa_boosters/parquet_full_ready_all_cols.pickle')

In [5]:
%%time

res_parquet, submit_df_parq = res_parquet_subm[res_parquet_subm['session_id'].notnull()], res_parquet_subm[res_parquet_subm['session_id'].isnull()]
submit_df_parq.drop(columns='session_id', inplace=True)

CPU times: user 16.1 s, sys: 20.7 s, total: 36.8 s
Wall time: 36.7 s


In [6]:
train_test_df, submit_df = df[df['session_id'].notnull()], df[df['session_id'].isnull()]

In [2457]:
%%time
train_test_df.set_index(['session_id', 'client_pin'], inplace=True)
res_parquet.set_index(['session_id', 'client_pin'], inplace=True)

CPU times: user 1min 9s, sys: 13.8 s, total: 1min 23s
Wall time: 1min 23s


In [2458]:
cols = [col for col in res_parquet.columns if col != 'multi_class_target']

In [2459]:
%%time
df_2 = res_parquet[cols].join(train_test_df, how='left')
train_test_df = df_2[df_2['multi_class_target'].notnull()]

CPU times: user 3min 38s, sys: 3min 57s, total: 7min 35s
Wall time: 7min 35s


In [2460]:
train_test_df.shape

(5065350, 655)

In [2461]:
%%time
del df_2
del res_parquet_subm
del res_parquet
del df

CPU times: user 9.5 s, sys: 10.7 s, total: 20.2 s
Wall time: 20.1 s


In [2462]:
import gc
gc.collect()

0

In [7]:
%%time
submit_df = pd.merge(submit_df, submit_df_parq, on=['client_pin'], how='left')

CPU times: user 9.18 s, sys: 8.97 s, total: 18.1 s
Wall time: 18.2 s


In [144]:
submit_df['last_target_parq_1']

0        credit_info
1              other
2        main_screen
3              other
4        main_screen
            ...     
79263          other
79264          other
79265          other
79266      statement
79267          other
Name: last_target_parq_1, Length: 79268, dtype: object

In [12]:
test_user = submit_df[submit_df['client_pin'] == 'e2f0cede9e3bc3d6ed7c77c4e08af136']

In [13]:
test_user[['timestamp', 'timestamp_first', 'timestamp_last']]

KeyError: "['timestamp_first', 'timestamp_last'] not in index"

In [ ]:
import gc
gc.collcet()

### TRAIN_TEST_SPLIT

In [2464]:
train_test_df.sort_values(by='timestamp', inplace=True)

In [2465]:
train_test_df.shape

(5065350, 655)

In [2466]:
# train_test_df = train_test_df[train_test_df['target_1'].notnull()]
# train_test_df = train_test_df[train_test_df['target_1']!='nan']
test_df = train_test_df.groupby('client_pin').tail(1)

In [2467]:
test_df.shape

(79268, 655)

In [2468]:
train_test_df.shape

(5065350, 655)

In [2469]:
train_df = train_test_df.sample(n=int(1e6), random_state=SEED)

In [2470]:
test_df['blank'] = '0'

In [19]:
import pickle

In [20]:
cat_atrs_parq = pickle.load(open('cat_atr_parq_clas.pickle','rb'))
lgbm_parq = pickle.load(open('session_parq_classifier.pickle','rb'))
atrs_parq = pickle.load(open('atr_parq_clas.pickle','rb'))

In [2477]:
%%time
train_df = train_df.merge(test_df['blank'], left_index=True, right_index=True, how='left', indicator=True)
train_df = train_df[train_df['_merge'] == 'left_only']

CPU times: user 2.44 s, sys: 2.91 s, total: 5.36 s
Wall time: 5.36 s


In [2478]:
f1_score(train_df['multi_class_target'], train_df['target_1'], average='macro')

0.3632993152191384

In [2480]:
%%time

train_df.to_pickle('/data/edmitrie/alfa_boosters/train_df_1kk.pickle')
test_df.to_pickle('/data/edmitrie/alfa_boosters/test_df.pickle')
submit_df.to_pickle('/data/edmitrie/alfa_boosters/submit_df.pickle')

CPU times: user 17.3 s, sys: 11.1 s, total: 28.4 s
Wall time: 28.7 s


In [74]:
atrs_to_drop = ['session_id','parquet_name', 'client_pin', 'multi_class_target', 'label', '_merge', 'month','blank','predicted_target'] + [col for col in train_df_less.columns if 'timestamp' in col] + atrs_parq

In [75]:
cat_atrs = ['day', 'weekday', 'hour'] + [col for col in train_df_less.columns if 'target' in col and 'multi' not in col and 'seq' not in col] + ['timezone', 'timezone_first', 'timezone_last'] + [col for col in train_df_less.columns if ('event' in col) or ('page' in col) or ('device' in col) or ('net_connection' in col) or ('application') in col or ('timezone') in col]

In [76]:
cat_atrs = [cat for cat in cat_atrs if cat in train_df.columns and 'mean' not in cat and 'sum' not in cat and 'num' not in cat and 'predicted' not in cat]

In [77]:
atrs = [atr for atr in train_df.columns if atr not in atrs_to_drop]

In [78]:
pickle.dump(cat_atrs, open('cat_atrs', 'wb'))
pickle.dump(atrs, open('atrs', 'wb'))

In [37]:
cat_atrs = pickle.load(open('cat_atrs', 'rb'))
atrs = pickle.load(open('atrs', 'rb'))

In [38]:
cat_atrs

['day',
 'weekday',
 'hour',
 'last_target_parq_1',
 'top_target_parq_2',
 'predicted_target',
 'top_target_parq_5',
 'top_target_parq_4',
 'top_target_parq_3',
 'last_target_parq_3',
 'last_target_parq_2',
 'last_target_parq_5',
 'last_target_parq_4',
 'top_target_parq_1',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'top_target',
 'top_target_2',
 'top_target_3',
 'event_name_first_prev_1',
 'device_is_webview_last_prev_1',
 'net_connection_type_last_prev_2',
 'event_category_first_prev_1',
 'event_name_last_prev_2',
 'application_id_first_prev_1',
 'event_type_first_prev_1',
 'timezone_prev_2',
 'device_screen_name_last_prev_2',
 'event_name_last_prev_1',
 'event_type_first_prev_2',
 'device_screen_name_first_prev_1',
 'application_id_last_prev_1',
 'event_label_first_prev_2',
 'event_category_first_prev_2',
 'event_category_last_prev_2',
 'net_connection_type_last_prev_1',
 'event_label_last_prev

In [393]:
for cat in cat_atrs:
#     print(cat)
    train_df.loc[train_df[cat].isnull(), cat] = 'nan'
#     test_df.loc[test_df[cat].isnull(), cat] = 'nan'
#     submit_df.loc[submit_df[cat].isnull(), cat] = 'nan'

In [392]:
train_df = pd.read_pickle('/data/edmitrie/alfa_boosters/train_df_1kk.pickle')

In [2490]:
#балансировка
class_weights = (len(train_df)/ train_df.multi_class_target.value_counts() ).to_dict()

In [2491]:
%%time

train_df_less = pd.DataFrame()

for col in  train_df['multi_class_target'].unique():
    if col in ['credit_info', 'own_transfer', 'mobile_recharge', 'phone_money_transfer', 'card2card_transfer','chat','card_recharge', 'invest']:
        frac = 1
        n = None
    else:
        frac = None
        n = 100000
    
    if col == 'credit_info':
        frac = None
        n = 50000
    
    train_df_less = pd.concat([train_df[train_df['multi_class_target'] == col].sample(frac=frac, n=n), train_df_less])

CPU times: user 1min 9s, sys: 26.8 s, total: 1min 36s
Wall time: 1min 36s


In [2492]:
train_df_less.to_pickle('/data/edmitrie/alfa_boosters/train_df_less_ready.pickle')

In [2493]:
# train_df_less.merge(test_df, on='session_id', how='inner').shape

## Получение Антоновских признаков, не сработало

In [172]:
anton_atrs_df = pd.read_pickle('/data/edmitrie/alfa_boosters/train_submit_df_time_target_2021_01_03.pickle')

In [189]:
train_df_anton, sumbit_df_antnon = anton_atrs_df[anton_atrs_df['session_id'].notnull()], anton_atrs_df[anton_atrs_df['session_id'].isnull()]

In [215]:
atrs_windows = [val for val in anton_atrs_df.columns.values if 'D' in val or 'W' in val or 'H' in val or 'M' in val] 

In [221]:
atrs_windows

['W_cumavg_target_prev_1_card2card_transfer',
 'D_cumavg_target_prev_1_card2card_transfer',
 'H_cumavg_target_prev_1_card2card_transfer',
 'M10_cumavg_target_prev_1_card2card_transfer',
 'M5_cumavg_target_prev_1_card2card_transfer',
 'W_cumavg_target_prev_1_card_recharge',
 'D_cumavg_target_prev_1_card_recharge',
 'H_cumavg_target_prev_1_card_recharge',
 'M10_cumavg_target_prev_1_card_recharge',
 'M5_cumavg_target_prev_1_card_recharge',
 'W_cumavg_target_prev_1_chat',
 'D_cumavg_target_prev_1_chat',
 'H_cumavg_target_prev_1_chat',
 'M10_cumavg_target_prev_1_chat',
 'M5_cumavg_target_prev_1_chat',
 'W_cumavg_target_prev_1_credit_info',
 'D_cumavg_target_prev_1_credit_info',
 'H_cumavg_target_prev_1_credit_info',
 'M10_cumavg_target_prev_1_credit_info',
 'M5_cumavg_target_prev_1_credit_info',
 'W_cumavg_target_prev_1_invest',
 'D_cumavg_target_prev_1_invest',
 'H_cumavg_target_prev_1_invest',
 'M10_cumavg_target_prev_1_invest',
 'M5_cumavg_target_prev_1_invest',
 'W_cumavg_target_prev_1_

In [224]:
anton_cat_atrs = [ 'prev_top_target_W','prev_top_target_D','prev_top_target_H']

In [225]:
new_cat_atr = anton_cat_atrs + cat_atrs

In [216]:
train_df_anton.set_index(['session_id', 'client_pin'],inplace=True)

KeyError: "None of ['session_id', 'client_pin'] are in the columns"

In [218]:
train_df_less = train_df_less.join(train_df_anton[atrs_windows])

In [229]:
test_df = test_df.join(train_df_anton[atrs_windows])

In [226]:
new_atrs  = atrs + atrs_windows

### Применение lightgbm

In [3]:
train_df_less = pd.read_pickle('/data/edmitrie/alfa_boosters/train_df_less_ready.pickle')
test_df = pd.read_pickle('/data/edmitrie/alfa_boosters/test_df.pickle')

NameError: name 'pd' is not defined

In [201]:
def evalerror(y_true, y_pred):
    preds = np.reshape(y_pred, (y_true.shape[0], 10), order='F')
    preds = preds.argmax(axis = 1)
    f_score = f1_score(y_true, preds, average = 'macro')
    
    return 'f1_score', f_score, True

In [396]:
for cat_atr in cat_atrs:
    train_df_less[cat_atr] = train_df_less[cat_atr].astype('category')
    test_df[cat_atr] = test_df[cat_atr].astype('category')
    train_df[cat_atr] = train_df[cat_atr].astype('category')
#     submit_df[cat_atr] = submit_df[cat_atr].astype('category')

In [231]:
from lightgbm import LGBMClassifier

In [245]:
lgbm = LGBMClassifier(objective='multiclass', reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=320, n_jobs=60, random_state=SEED)

In [47]:
cols_bigger = [atr for atr in atrs if 'bigger' in atr]

In [399]:
for col in cols_bigger:
    train_df_less[col] = train_df_less[col].astype(np.float32)
    test_df[col] = test_df[col].astype(np.float32)
    train_df[col] = train_df[col].astype(np.float32)

In [236]:
atrs = [atr for atr in atrs if 'proba' not in atr or 'prev' in atr]

In [237]:
train_df_less.loc[train_df_less['timezone_num_prev_2'] == 'nan','timezone_num_prev_2'] = 0
train_df_less.loc[train_df_less['timezone_num_prev_1'] == 'nan','timezone_num_prev_1'] = 0


In [238]:
train_df_less['timezone_num_prev_2'] = train_df_less['timezone_num_prev_2'].astype(int)
train_df_less['timezone_num_prev_1'] = train_df_less['timezone_num_prev_1'].astype(int)

In [271]:
from itertools import combinations

In [350]:
# не даёт буста!!!

targ_comb_cols = []

for combin in combinations(range(1, 5), 2):
    col = f'target_{"_".join(map(str, combin))}'
    train_df_less[col] = ''
    test_df[col] = ''
    
    targ_comb_cols.append(col)
    
    for el in combin:
        train_df_less[col] += train_df_less[f'target_{el}'].astype(str)
        test_df[col] += test_df[f'target_{el}'].astype(str)
    
    train_df_less[col] = train_df_less[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# for combin in combinations(range(1, 6), 3):
#     col = f'target_{"_".join(map(str, combin))}'
#     train_df_less[col] = ''
#     test_df[col] = ''
    
#     targ_comb_cols.append(col)
    
#     for el in combin:
#         train_df_less[col] += train_df_less[f'target_{el}'].astype(str)
#         test_df[col] += test_df[f'target_{el}'].astype(str)
    
#     train_df_less[col] = train_df_less[col].astype('category')
#     test_df[col] = test_df[col].astype('category')
    
    


In [354]:
train_test_split(test_size = 0.5)

['top_target',
 'target_1',
 'top_target_2',
 'prev_sum_parq_own_transfer',
 'prev_sum_own_transfer',
 'bigger_credit_info_card2card_transfer',
 'sessions_number',
 'bigger_card_recharge_card2card_transfer',
 'seq_last_target_len',
 'bigger_mobile_recharge_chat',
 'prev_sum_parq_card2card_transfer',
 'prev_sum_credit_info',
 'hour',
 'prev_sum_phone_money_transfer',
 'device_screen_name_last_prev_1',
 'day',
 'prev_sum_statement',
 'target_2',
 'prev_sum_card2card_transfer',
 'prev_sum_card_recharge',
 'device_screen_name_Chat_mean_prev_1',
 'target_3',
 'prev_sum_parq_phone_money_transfer',
 'bigger_phone_money_transfer_chat',
 'target_4',
 'device_screen_name_last_prev_2',
 'timezone_prev_1',
 'top_target_3',
 'timezone_prev_2',
 'target_5',
 'prev_sum_parq_mobile_recharge',
 'mean_diff_time_global_explotation',
 'prev_sum_parq_invest',
 'prev_sum_parq_card_recharge',
 'target_6',
 'time_diff_1_0',
 'prev_sum_invest',
 'target_7',
 'prev_sum_parq_statement',
 'device_screen_name_Stat

In [402]:
lgbm.fit(train_df_less[new_atrs], train_df_less['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.60937	valid_0's f1_score: 0.393445
[10]	valid_0's multi_logloss: 1.50766	valid_0's f1_score: 0.405807
[15]	valid_0's multi_logloss: 1.46878	valid_0's f1_score: 0.407204
[20]	valid_0's multi_logloss: 1.4507	valid_0's f1_score: 0.407275
[25]	valid_0's multi_logloss: 1.44223	valid_0's f1_score: 0.408705
[30]	valid_0's multi_logloss: 1.43929	valid_0's f1_score: 0.407893
[35]	valid_0's multi_logloss: 1.43594	valid_0's f1_score: 0.407413
[40]	valid_0's multi_logloss: 1.43478	valid_0's f1_score: 0.4077
[45]	valid_0's multi_logloss: 1.43294	valid_0's f1_score: 0.407375
[50]	valid_0's multi_logloss: 1.43161	valid_0's f1_score: 0.407702
[55]	valid_0's multi_logloss: 1.43054	valid_0's f1_score: 0.407723
[60]	valid_0's multi_logloss: 1.42936	valid_0's f1_score: 0.407248
[65]	valid_0's multi_logloss: 1.42847	valid_0's f1_score: 0.407836
[70]	valid_0's multi_logloss: 1.42801	valid_0's f1_score: 0.408551
[75]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=320, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [543]:
train_df_sampled, ecode_ftr_df = train_test_split(train_df_less, test_size = 0.5, stratify=train_df_less['multi_class_target'],random_state=SEED + 100)

In [544]:
lgbm.fit(ecode_ftr_df[new_atrs], ecode_ftr_df['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.61301	valid_0's f1_score: 0.392908
[10]	valid_0's multi_logloss: 1.50982	valid_0's f1_score: 0.404058
[15]	valid_0's multi_logloss: 1.4699	valid_0's f1_score: 0.404497
[20]	valid_0's multi_logloss: 1.45395	valid_0's f1_score: 0.405766
[25]	valid_0's multi_logloss: 1.44582	valid_0's f1_score: 0.4057
[30]	valid_0's multi_logloss: 1.44381	valid_0's f1_score: 0.405312
[35]	valid_0's multi_logloss: 1.44	valid_0's f1_score: 0.40543
[40]	valid_0's multi_logloss: 1.43856	valid_0's f1_score: 0.405766
[45]	valid_0's multi_logloss: 1.43657	valid_0's f1_score: 0.40579
[50]	valid_0's multi_logloss: 1.435	valid_0's f1_score: 0.406111
[55]	valid_0's multi_logloss: 1.43428	valid_0's f1_score: 0.405849
[60]	valid_0's multi_logloss: 1.43294	valid_0's f1_score: 0.406398
[65]	valid_0's 

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=320, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [548]:
buf = pd.DataFrame(lgbm.predict_proba(train_df_sampled[new_atrs]), index=train_df_sampled.index)

In [549]:
buf_cols = buf.columns.values.tolist()

In [550]:
train_df_sampled_all = pd.concat([buf, train_df_sampled], axis=1)

In [554]:
test_df_all = pd.concat([pd.DataFrame(lgbm.predict_proba(test_df[new_atrs]), index=test_df.index), test_df], axis=1)

In [551]:
lgbm_2 = LGBMClassifier(objective='multiclass', reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=320, n_jobs=60, random_state=SEED)

In [555]:
lgbm_2.fit(train_df_sampled_all[new_atrs + buf_cols], train_df_sampled_all['multi_class_target'], eval_set=(test_df_all[new_atrs + buf_cols], test_df_all['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.58647	valid_0's f1_score: 0.409215
[10]	valid_0's multi_logloss: 1.48744	valid_0's f1_score: 0.410673
[15]	valid_0's multi_logloss: 1.45208	valid_0's f1_score: 0.409374
[20]	valid_0's multi_logloss: 1.43853	valid_0's f1_score: 0.408335
[25]	valid_0's multi_logloss: 1.43286	valid_0's f1_score: 0.407783
[30]	valid_0's multi_logloss: 1.43017	valid_0's f1_score: 0.407265
[35]	valid_0's multi_logloss: 1.42981	valid_0's f1_score: 0.407446
[40]	valid_0's multi_logloss: 1.42993	valid_0's f1_score: 0.407085
[45]	valid_0's multi_logloss: 1.42986	valid_0's f1_score: 0.406423
[50]	valid_0's multi_logloss: 1.42947	valid_0's f1_score: 0.406118
[55]	valid_0's multi_logloss: 1.42959	valid_0's f1_score: 0.406288
[60]	valid_0's multi_logloss: 1.42999	valid_0's f1_score: 0.406151
[65]	

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=320, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [369]:
def get_new_features(df):
        for i in range(df.shape[0]):
            if df['im'].iloc[:i].sum()/df['im'].sum() > 0.99:
                break
        return df['f'].iloc[:i].values.tolist()

In [370]:
f_imp = pd.DataFrame(zip(lgbm_orig.feature_name_, lgbm_orig.feature_importances_), columns=['f', 'im']).sort_values(by='im', ascending=False)

In [371]:
new_atrs = get_new_features(f_imp)

In [372]:
lgbm.fit(train_df_less[new_atrs], train_df_less['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.60937	valid_0's f1_score: 0.393445
[10]	valid_0's multi_logloss: 1.50766	valid_0's f1_score: 0.405807
[15]	valid_0's multi_logloss: 1.46878	valid_0's f1_score: 0.407204
[20]	valid_0's multi_logloss: 1.4507	valid_0's f1_score: 0.407275
[25]	valid_0's multi_logloss: 1.44223	valid_0's f1_score: 0.408705
[30]	valid_0's multi_logloss: 1.43929	valid_0's f1_score: 0.407893
[35]	valid_0's multi_logloss: 1.43594	valid_0's f1_score: 0.407413
[40]	valid_0's multi_logloss: 1.43478	valid_0's f1_score: 0.4077
[45]	valid_0's multi_logloss: 1.43294	valid_0's f1_score: 0.407375
[50]	valid_0's multi_logloss: 1.43161	valid_0's f1_score: 0.407702
[55]	valid_0's multi_logloss: 1.43054	valid_0's f1_score: 0.407723
[60]	valid_0's multi_logloss: 1.42936	valid_0's f1_score: 0.407248
[65]	valid_0's multi_logloss: 1.42847	valid_0's f1_score: 0.407836
[70]	valid_0's multi_logloss: 1.42801	valid_0's f1_score: 0.408551
[75]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=320, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

## hyperopt для class_weight

In [645]:
from hyperopt import hp, tpe, Trials, space_eval
from hyperopt.fmin import fmin

In [621]:
labels = test_df['multi_class_target'].unique()

In [638]:
def get_loss_model_hp(params):
    lgbm = LGBMClassifier(objective='multiclass', **params, reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=320, n_jobs=60, random_state=SEED, )
    
    lgbm.fit(train_df_less[new_atrs], train_df_less['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=False, early_stopping_rounds=100)
    
    metric = f1_score(test_df['multi_class_target'], lgbm.predict(test_df[new_atrs]), average='macro')
          
    print(datetime.datetime.now())
    print(params, metric)
    
    return -metric

In [639]:
par_space_catboost = {
      'class_weight':  {el:hp.uniform(el, 0, 1)  for el in labels}
}

In [640]:
top_targets = [atr for atr in new_atrs if 'top' in atr]

In [641]:
rstate = np.random.RandomState(SEED) 

In [5]:
# trial = Trials()
# best = fmin(fn=get_loss_model_hp, space=par_space_catboost, algo=tpe.suggest, max_evals=300, rstate=rstate) 

In [644]:
best

{'card2card_transfer': 0.6398475182669389,
 'card_recharge': 0.4502000735911028,
 'chat': 0.3815013547758437,
 'credit_info': 0.46101125727352393,
 'invest': 0.9321761812255462,
 'main_screen': 0.982994897893273,
 'mobile_recharge': 0.641212783884727,
 'own_transfer': 0.6317591535575899,
 'phone_money_transfer': 0.4600662111029351,
 'statement': 0.6951657522904898}

In [652]:
evaled = space_eval(par_space_catboost, best)

In [654]:
lgbm = LGBMClassifier(objective='multiclass', class_weight = evaled['class_weight'], reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=320, n_jobs=60, random_state=SEED, )

In [6]:
# lgbm.fit(train_df_less[new_atrs], train_df_less['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

In [656]:
f1_score(test_df['multi_class_target'], lgbm.predict(test_df[new_atrs]), average='macro')

0.4147348802843952

In [404]:
labels = test_df['multi_class_target'].unique()

In [405]:
conf_matr_df = pd.DataFrame(confusion_matrix(test_df['multi_class_target'], lgbm.predict(test_df[new_atrs]), labels), columns=labels, index=labels)

In [386]:
conf_matr_df

,card_recharge,main_screen,credit_info,statement,phone_money_transfer,card2card_transfer,own_transfer,mobile_recharge,invest,chat
card_recharge,1641,431,492,283,51,48,62,43,9,149
main_screen,1869,18665,3059,4321,1498,830,1247,1353,83,1152
credit_info,570,1267,6873,672,116,148,298,152,17,535
statement,592,2315,935,5972,824,393,630,695,41,539
phone_money_transfer,119,745,148,722,1411,199,341,299,16,129
card2card_transfer,98,464,178,343,206,797,171,189,4,93
own_transfer,76,349,228,334,218,90,1800,147,29,124
mobile_recharge,83,497,158,547,304,184,236,1257,4,67
invest,45,155,58,131,43,14,78,25,246,66
chat,239,834,531,641,133,90,202,123,22,1318


In [382]:
np.sum(test_df['multi_class_target'] == 'invest')

861

In [385]:
conf_matr_df.loc['invest'].sum()

861

In [ ]:
 lgbm.predict(test_df[new_atrs]), labels

In [582]:
res_probs_df = pd.DataFrame(lgbm.predict_proba(test_df[new_atrs]), columns=lgbm.classes_)

In [583]:
prob_cols = res_probs_df.columns.values

In [584]:
prob_cols

array(['card2card_transfer', 'card_recharge', 'chat', 'credit_info',
       'invest', 'main_screen', 'mobile_recharge', 'own_transfer',
       'phone_money_transfer', 'statement'], dtype=object)

In [585]:
res_probs_df['target'] = test_df.reset_index()['multi_class_target']

In [586]:
res_probs_df['predicted_1'] = prob_cols[res_probs_df[prob_cols].values.argsort(1)[:, -1]]
res_probs_df['predicted_2'] = prob_cols[res_probs_df[prob_cols].values.argsort(1)[:, -2]]
# res_probs_df['predicted_3'] = prob_cols[res_probs_df[prob_cols].values.argsort(1)[:, -3]]
# res_probs_df['predicted_4'] = prob_cols[res_probs_df[prob_cols].values.argsort(1)[:, -4]]

In [564]:
res_probs_df[(res_probs_df['target'] == 'invest') & (res_probs_df['predicted_1'] == 'credit_info')].shape

(58, 16)

In [588]:
f1_score(res_probs_df['target'], res_probs_df['predicted_1'], average='macro')

0.40815365709638673

In [567]:
pd.DataFrame(classification_report(test_df['multi_class_target'], lgbm.predict(test_df[new_atrs]), output_dict=True)).T

,precision,recall,f1-score,support
card2card_transfer,0.283006,0.316948,0.299017,2543.000000
card_recharge,0.329812,0.476784,0.389908,3209.000000
chat,0.307180,0.312606,0.309869,4133.000000
credit_info,0.535668,0.656555,0.589983,10648.000000
invest,0.551724,0.278746,0.370370,861.000000
main_screen,0.720402,0.553541,0.626043,34077.000000
mobile_recharge,0.285747,0.368894,0.322041,3337.000000
own_transfer,0.344714,0.523417,0.415673,3395.000000
phone_money_transfer,0.288925,0.348753,0.316032,4129.000000
statement,0.434017,0.451531,0.442601,12936.000000


In [608]:
res_probs_df['predicted'] = res_probs_df['predicted_1']

In [598]:
# res_probs_df.loc[(res_probs_df['predicted'] == 'credit_info') &  (res_probs_df['predicted_2'] == 'invest'),  'predicted'] = 'invest'
# res_probs_df.loc[(res_probs_df['predicted'] == 'own_transfer') &  (res_probs_df['predicted_2'] == 'invest'),  'predicted'] = 'invest'
# res_probs_df.loc[(res_probs_df['predicted'] == 'card_recharge') &  (res_probs_df['predicted_2'] == 'invest'),  'predicted'] = 'invest'
# res_probs_df.loc[(res_probs_df['predicted'] == 'phone_money_transfer') &  (res_probs_df['predicted_2'] == 'invest'),  'predicted'] = 'invest'
# res_probs_df.loc[(res_probs_df['predicted'] == 'statement') &  (res_probs_df['predicted_2'] == 'invest'), 'predicted'] = 'invest'
# res_probs_df.loc[(res_probs_df['predicted_1'] == 'mobile_recharge') &  (res_probs_df['predicted_2'] == 'invest'), 'predicted'] = 'invest'

In [614]:
for el in  prob_cols:
    buf = res_probs_df[(res_probs_df['predicted_1'] == 'phone_money_transfer') & (res_probs_df['target'] == el)]
    print(el, np.mean(buf['target'] == buf['predicted_2']))

card2card_transfer 0.3
card_recharge 0.18032786885245902
chat 0.07741935483870968
credit_info 0.1937984496124031
invest 0.0425531914893617
main_screen 0.44805608667941366
mobile_recharge 0.3259493670886076
own_transfer 0.5022222222222222
phone_money_transfer 0.0
statement 0.5216346153846154


In [615]:
res_probs_df.loc[(res_probs_df['predicted'] == 'phone_money_transfer') &  (res_probs_df['predicted_2'] == 'own_transfer'),  'predicted'] = 'own_transfer'

In [616]:
f1_score(res_probs_df['target'], res_probs_df['predicted'], average='macro')

0.39634841878425087

In [581]:
f1_score(res_probs_df['target'], res_probs_df['predicted'], average='macro')

0.37317628783257867

In [540]:
pd.DataFrame(classification_report(res_probs_df['target'], res_probs_df['predicted'], output_dict=True)).T

,precision,recall,f1-score,support
card2card_transfer,0.285356,0.313409,0.298726,2543.000000
card_recharge,0.307764,0.511374,0.384264,3209.000000
chat,0.315916,0.318897,0.317399,4133.000000
credit_info,0.542891,0.645473,0.589755,10648.000000
invest,0.522293,0.285714,0.369369,861.000000
main_screen,0.725643,0.547730,0.624258,34077.000000
mobile_recharge,0.293486,0.376686,0.329921,3337.000000
own_transfer,0.355380,0.530191,0.425532,3395.000000
phone_money_transfer,0.293714,0.341729,0.315907,4129.000000
statement,0.427610,0.461657,0.443982,12936.000000


In [510]:
f1_score(res_probs_df['target'], res_probs_df['predicted'], average='macro')

0.409177243509488

In [453]:
pd.DataFrame(classification_report(res_probs_df['target'], res_probs_df['predicted'], output_dict=True)).T

,precision,recall,f1-score,support
card2card_transfer,0.285356,0.313656,0.298838,2541.000000
card_recharge,0.307678,0.511229,0.384156,3206.000000
chat,0.315916,0.319670,0.317782,4123.000000
credit_info,0.543024,0.645361,0.589787,10639.000000
invest,0.610727,0.371970,0.462344,949.000000
main_screen,0.725643,0.547987,0.624425,34061.000000
mobile_recharge,0.293486,0.376912,0.330008,3335.000000
own_transfer,0.353954,0.526034,0.423169,3361.000000
phone_money_transfer,0.293860,0.341174,0.315754,4124.000000
statement,0.427610,0.461907,0.444097,12929.000000


In [ ]:
submit_df['prediction'] = lgbm.predict(submit_df)

In [465]:
submit_df.set_index('client_pin', inplace=True)

In [491]:
res_probs_df_submit = pd.DataFrame(lgbm.predict_proba(submit_df[new_atrs]), index=submit_df.index, columns=lgbm.classes_,)

In [ ]:
res_probs_df_submit

In [362]:
lgbm.fit(target_encoder.transform(train_df_less[new_atrs + targ_comb_cols]), train_df_less['multi_class_target'], eval_set=(target_encoder.transform(test_df[new_atrs + targ_comb_cols]), test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.60771	valid_0's f1_score: 0.398017
[10]	valid_0's multi_logloss: 1.50591	valid_0's f1_score: 0.405514
[15]	valid_0's multi_logloss: 1.46681	valid_0's f1_score: 0.407278
[20]	valid_0's multi_logloss: 1.45163	valid_0's f1_score: 0.40704
[25]	valid_0's multi_logloss: 1.44176	valid_0's f1_score: 0.408143
[30]	valid_0's multi_logloss: 1.43798	valid_0's f1_score: 0.40788
[35]	valid_0's multi_logloss: 1.43603	valid_0's f1_score: 0.407757
[40]	valid_0's multi_logloss: 1.43358	valid_0's f1_score: 0.407727
[45]	valid_0's multi_logloss: 1.43209	valid_0's f1_score: 0.408504
[50]	valid_0's multi_logloss: 1.4313	valid_0's f1_score: 0.408956
[55]	valid_0's multi_logloss: 1.43056	valid_0's f1_score: 0.408694
[60]	valid_0's multi_logloss: 1.429	valid_0's f1_score: 0.409094
[65]	valid_0's multi_logloss: 1.42851	valid_0's f1_score: 0.409408
[70]	valid_0's multi_logloss: 1.42773	valid_0's f1_score: 0.410236
[75]	v

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=320, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [326]:
def get_new_features(df):
        for i in range(df.shape[0]):
            if df['im'].iloc[:i].sum()/df['im'].sum() > 0.99:
                break
        return df['f'].iloc[:i].values.tolist()

In [343]:
from category_encoders.wrapper import PolynomialWrapper
from category_encoders import *

def get_target_encoder(df, cols, target):
    enc = TargetEncoder(cols=cols)
    wrapper = PolynomialWrapper(enc)
    
    wrapper.fit(df, target)
    
    return wrapper

In [351]:
train_df_sampled, ecode_ftr_df = train_test_split(train_df_less, test_size = 0.2, stratify=train_df_less['multi_class_target'],random_state=SEED + 100)

In [360]:
target_encoder = get_target_encoder(ecode_ftr_df[new_atrs + targ_comb_cols], targ_comb_cols + target_atrs, ecode_ftr_df['multi_class_target'])

In [ ]:
# train_df_sampled, ecode_ftr_df = train_test_split(train_df_less, test_size = 0.2, stratify=train_df_less['multi_class_target'],random_state=SEED + 100)

In [366]:
classification_report(test_df['multi_class_target'], target_encoder.transform(lgbm.predict(target_encoder.transform(test_df[new_atrs])), output_dict=True)

ValueError: Number of features of the model must match the input. Model n_features_ is 458 and input n_features is 212 

In [59]:
lgbm_orig = lgbm

In [60]:
f_imp = pd.DataFrame(zip(lgbm_orig.feature_name_, lgbm_orig.feature_importances_), columns=['f', 'im']).sort_values(by='im', ascending=False)

In [61]:
# pd.options.display.max_rows = 1000

In [2515]:
f_imp.head(n=120)[0].values

KeyError: 0

In [62]:
def get_new_features(df):
        for i in range(df.shape[0]):
            if df['im'].iloc[:i].sum()/df['im'].sum() > 0.9999:
                break
        return df['f'].iloc[:i].values.tolist()

In [63]:
new_atrs = get_new_features(f_imp)

In [2525]:
lgbm.fit(train_df_less[new_atrs], train_df_less['multi_class_target'], eval_set=(test_df[new_atrs], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.60939	valid_0's f1_score: 0.392608
[10]	valid_0's multi_logloss: 1.50724	valid_0's f1_score: 0.404997
[15]	valid_0's multi_logloss: 1.46709	valid_0's f1_score: 0.406296
[20]	valid_0's multi_logloss: 1.44798	valid_0's f1_score: 0.406961
[25]	valid_0's multi_logloss: 1.43817	valid_0's f1_score: 0.407399
[30]	valid_0's multi_logloss: 1.43185	valid_0's f1_score: 0.407598
[35]	valid_0's multi_logloss: 1.429	valid_0's f1_score: 0.407719
[40]	valid_0's multi_logloss: 1.42665	valid_0's f1_score: 0.40773
[45]	valid_0's multi_logloss: 1.4249	valid_0's f1_score: 0.407826
[50]	valid_0's multi_logloss: 1.4237	valid_0's f1_score: 0.407938
[55]	valid_0's multi_logloss: 1.42302	valid_0's f1_score: 0.408715
[60]	valid_0's multi_logloss: 1.42221	valid_0's f1_score: 0.408544
[65]	valid_0's multi_logloss: 1.42176	valid_0's f1_score: 0.408279
[70]	valid_0's multi_logloss: 1.4217	valid_0's f1_score: 0.408532
[75]	va

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=250, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [2650]:
cat_atrs = [atr for atr in cat_atrs if atr in atrs]

In [64]:
# train_df_less['timezone_num_prev_1'] = train_df_less['timezone_num_prev_1'].astype(np.float)
# test_df['timezone_num_prev_1'] = test_df['timezone_num_prev_1'].astype(np.float)

In [86]:
for atr in cat_atrs:
    train_df_less[atr] = train_df_less[atr].astype('object')
    train_df_less[atr].fillna('nan', inplace=True)
    
    test_df[atr] = test_df[atr].astype('object')
    test_df[atr].fillna('nan', inplace=True)
    
    if str(test_df[atr].dtype) == 'float64':
        train_df_less[atr] =   train_df_less[atr].astype(int)
        test_df[atr] =   test_df[atr].astype(int)

In [99]:
cv_clsf = cb.CatBoostClassifier(objective='MultiClass',custom_metric=['F1'], verbose=True, eval_metric='TotalF1:average=Macro', random_seed=SEED, n_estimators=550, one_hot_max_size=100, learning_rate=0.4)

In [100]:
cv_clsf.fit(train_df_less[atrs], train_df_less['multi_class_target'], eval_set=(test_df[atrs], test_df['multi_class_target']), verbose=4, use_best_model=True, cat_features= cat_atrs, early_stopping_rounds=120)

0:	learn: 0.2703405	test: 0.2345578	best: 0.2345578 (0)	total: 2.44s	remaining: 22m 20s
4:	learn: 0.4453213	test: 0.3676144	best: 0.3676144 (4)	total: 11.4s	remaining: 20m 42s
8:	learn: 0.4615654	test: 0.3787277	best: 0.3787277 (8)	total: 20.7s	remaining: 20m 46s
12:	learn: 0.4700501	test: 0.3863819	best: 0.3863819 (12)	total: 29.5s	remaining: 20m 20s
16:	learn: 0.4740463	test: 0.3867700	best: 0.3873080 (14)	total: 38.4s	remaining: 20m 4s
20:	learn: 0.4770293	test: 0.3890505	best: 0.3890505 (20)	total: 47.4s	remaining: 19m 54s
24:	learn: 0.4794592	test: 0.3902320	best: 0.3902320 (24)	total: 56.5s	remaining: 19m 46s
28:	learn: 0.4808927	test: 0.3914399	best: 0.3914399 (28)	total: 1m 4s	remaining: 19m 26s
32:	learn: 0.4830589	test: 0.3940380	best: 0.3940380 (32)	total: 1m 13s	remaining: 19m 15s
36:	learn: 0.4850002	test: 0.3951433	best: 0.3951433 (36)	total: 1m 21s	remaining: 18m 55s
40:	learn: 0.4865550	test: 0.3962442	best: 0.3962442 (40)	total: 1m 31s	remaining: 18m 52s
44:	learn: 0.4

KeyboardInterrupt: 

In [148]:
def get_model_est_features_enc(model, metric, X, y, X_eval, y_eval):
    k_fold = StratifiedKFold(5, shuffle=True, random_state=SEED)
    metrics = []
    models = []

    for train_ind, test_ind in k_fold.split(X, y):
        X_train, X_test = X.iloc[train_ind], X.iloc[test_ind]
        y_train, y_test = y.iloc[train_ind], y.iloc[test_ind]
        
        model = LGBMClassifier(objective='multiclass', reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=250, n_jobs=60, random_state=SEED)

        model.fit(X_train, y_train, eval_set = (X_eval, y_eval), eval_metric=evalerror, early_stopping_rounds=100)
        models.append(model)

        metrics.append(metric(y_eval, model.predict(X_eval), average='macro'))
        print(metrics[-1])

    return models, metrics

In [149]:
models, metrics = get_model_est_features_enc(lgbm, f1_score, train_df_less[atrs], train_df_less['multi_class_target'], test_df[atrs], test_df['multi_class_target'])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's multi_logloss: 1.84164	valid_0's f1_score: 0.201584
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.75707	valid_0's f1_score: 0.330918
[3]	valid_0's multi_logloss: 1.6956	valid_0's f1_score: 0.371405
[4]	valid_0's multi_logloss: 1.64853	valid_0's f1_score: 0.386328
[5]	valid_0's multi_logloss: 1.61169	valid_0's f1_score: 0.392767
[6]	valid_0's multi_logloss: 1.58134	valid_0's f1_score: 0.396712
[7]	valid_0's multi_logloss: 1.55749	valid_0's f1_score: 0.399528
[8]	valid_0's multi_logloss: 1.53789	valid_0's f1_score: 0.401999
[9]	valid_0's multi_logloss: 1.52192	valid_0's f1_score: 0.403173
[10]	valid_0's multi_logloss: 1.50875	valid_0's f1_score: 0.404285
[11]	valid_0's multi_logloss: 1.49777	valid_0's f1_score: 0.404747
[12]	valid_0's multi_logloss: 1.48893	valid_0's f1_score: 0.404721
[13]	valid_0's

In [167]:
predicts = models[0].predict_proba(submit_df[atrs])

In [168]:
for i in range(1, 5):
    print(predicts.shape)
    predicts += models[i].predict_proba(submit_df[atrs])

(79268, 10)
(79268, 10)
(79268, 10)
(79268, 10)


In [170]:
submit_df['prediction'] = models[0].classes_[predicts.argmax(axis=-1)]

In [171]:
submit_df[['client_pin', 'prediction']].to_csv('ensemble.csv', index=False)

### Линейная регрессия

In [2526]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

import pickle

In [2527]:
# clf = LogisticRegression()

In [2528]:
dict_cols_categ = pickle.load(open('dict_usefull_ctgrs','rb'))

In [2529]:
from category_encoders.wrapper import PolynomialWrapper
from category_encoders import *

def get_target_encoder(df, cols, target):
    enc = TargetEncoder(cols=cols_to_encode)
    wrapper = PolynomialWrapper(enc)
    
    wrapper.fit(ecode_ftr_df, ecode_ftr_df[target])
    
    return wrapper

In [2534]:
target_encoding = [atr for atr in cat_atrs if 'prev' in atr]

In [2532]:
# bigger_cols = [col for col in df.columns if 'bigger' in col]

In [ ]:
prev_sum_cols = [col for col in df.columns.values if 'prev_sum' in col]

In [ ]:
df.columns.values

In [2307]:
train_df_atrs = [atr for atr in atrs if atr in df.columns.values]

In [275]:
mean_prev_ftrs = [atr for atr in atrs if 'mean_prev' in atr]

In [277]:
len(mean_prev_ftrs)

414

In [1747]:
last_prev_atrs = [atr for atr in last_prev_atrs if atr not in ['page_urlhost_last_prev_1','device_is_webview_last_prev_1','page_urlpath_full_last_prev_1']]

In [2412]:
train_df_atrs

['bigger_mobile_recharge_statement',
 'bigger_mobile_recharge_phone_money_transfer',
 'bigger_mobile_recharge_chat',
 'bigger_mobile_recharge_invest',
 'bigger_mobile_recharge_main_screen',
 'bigger_mobile_recharge_own_transfer',
 'bigger_mobile_recharge_card_recharge',
 'bigger_mobile_recharge_credit_info',
 'bigger_mobile_recharge_card2card_transfer',
 'bigger_statement_phone_money_transfer',
 'bigger_statement_chat',
 'bigger_statement_invest',
 'bigger_statement_main_screen',
 'bigger_statement_own_transfer',
 'bigger_statement_card_recharge',
 'bigger_statement_credit_info',
 'bigger_statement_card2card_transfer',
 'bigger_phone_money_transfer_chat',
 'bigger_phone_money_transfer_invest',
 'bigger_phone_money_transfer_main_screen',
 'bigger_phone_money_transfer_own_transfer',
 'bigger_phone_money_transfer_card_recharge',
 'bigger_phone_money_transfer_credit_info',
 'bigger_phone_money_transfer_card2card_transfer',
 'bigger_chat_invest',
 'bigger_chat_main_screen',
 'bigger_chat_ow

In [2262]:
# cols = ['target_1','top_target','target_2','top_target_2','top_target_3','target_3', 'target_4','target_5','target_6', 'target_7','target_8','target_9','mean_diff_time_global', 'seq_last_target_len','day', 'weekday','hour','device_screen_name_last_prev_1','device_screen_name_last_prev_2','last_target_parq_1', 'last_target_parq_2','event_label_last_prev_1','timezone_prev_1','top_target_parq_1','time_diff_1_0','mean_diff_time_global_explotation','timezone_num_prev_1', 'time_diff_1_0_parq'] + prev_sum_cols + mean_prev_ftrs 

cols = list(set(cols))

In [2290]:
cols = ['target_1','top_target','target_2','top_target_2','top_target_3','target_3', 'target_4','target_5','target_6', 'target_7','target_8','target_9','mean_diff_time_global', 'seq_last_target_len','day','weekday','hour','time_diff_1_0','mean_diff_time_global_explotation'] + bigger_cols #'last_target_parq_1','device_screen_name_last_prev_1']

cols = list(set(cols))

cols = train_df_atrs

In [2310]:
# cat_cols = ['target_1', 'top_target','target_2','top_target_2','top_target_3','target_3','target_4','target_5','target_6', 'target_7','target_8', 'target_9','day', 'weekday','hour','last_target_parq_1','last_target_parq_2',]

In [2311]:
cat_cols = ['target_1', 'top_target','target_2','top_target_2','top_target_3','target_3','target_4','target_5','target_6', 'target_7','target_8', 'target_9','day', 'weekday','hour']

In [2312]:
train_df_lin = train_df_less.reset_index()[cols + ['multi_class_target', 'session_id', 'client_pin','timestamp']]
test_df_lin = test_df.reset_index()[cols + ['multi_class_target','session_id','client_pin', 'timestamp']]
submit_df_lin = submit_df[cols + ['client_pin']].reset_index(drop=True)

train_df_lin['train'] = 1
test_df_lin['train'] = 0
submit_df_lin['train'] = -1

In [2313]:
df_lin = pd.concat([train_df_lin, test_df_lin, submit_df_lin])

In [2314]:
# df_lin['timezone_num_prev_1'] = df_lin['timezone_num_prev_1'].astype(int)
# df_lin['timezone_num_prev_2'] = df_lin['timezone_num_prev_2'].astype(int)

In [2315]:
df_lin = pd.get_dummies(df_lin, columns=cat_cols)

In [2316]:
# df_lin['day'] = df_lin['day'].astype(np.float)
# df_lin['weekday'] = df_lin['weekday'].astype(np.float)
# df_lin['hour'] = df_lin['hour'].astype(np.float)

In [2317]:
# cols_to_encode = ['device_screen_name_last_prev_1','device_screen_name_last_prev_2','event_label_last_prev_1','timezone_prev_1','top_target_parq_1'] #+ last_prev_atrs
cols_to_encode = []#['device_screen_name_last_prev_1','page_urlpath_full_last_prev_1']   #last_prev_atrs + ['device_screen_name_last_prev_1','page_urlpath_full_last_prev_1'] 
cols_to_encode = list(set(cols_to_encode))

In [2318]:
for col in cols_to_encode:
    for key in dict_cols_categ:
        if key in col:
            print(col)
            df_lin.loc[df_lin[col].apply(lambda x: x not in dict_cols_categ[key]), col] = 'other'

In [2319]:
for col in cols_to_encode:
    df_lin[col] = df_lin[col].astype('category')

In [2320]:
train_df_lin, test_df_lin, submit_df_lin = df_lin[df_lin['train'] == 1], df_lin[df_lin['train'] == 0], df_lin[df_lin['train'] == -1]

In [2321]:
train_df_sampled, ecode_ftr_df = train_test_split(train_df_lin, test_size = 0.2, stratify=train_df_lin['multi_class_target'],random_state=SEED + 100)

In [2323]:
cols = [col for col in train_df_sampled.columns if col not in ['multi_class_target', 'train','client_pin','session_id','timestamp']]

In [2324]:
# clf = LogisticRegression()

In [2325]:
# %%time

# clf.fit(train_df_sampled[cols].fillna(0), train_df_sampled['multi_class_target'])

In [2326]:
# f1_score(lb_target.transform(test_df_lin['multi_class_target']), clf_lasso.predict(test_df_lin[cols].fillna(0)), average='macro')

In [2327]:
# f1_score(test_df_lin['multi_class_target'], clf.predict(test_df_lin[cols]), average='macro')

In [2328]:
# lgbm = LGBMClassifier(objective='multiclass', reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=200, n_jobs=60, random_state=SEED)

In [2426]:
lgbm = LGBMClassifier(objective='multiclass', reg_alpha=12, max_depth=10, min_split_gain = 0.7, subsample=0.7, n_estimators=210, n_jobs=64, random_state=SEED)

In [2427]:
train_df_sampled.sort_values(by='timestamp', inplace=True)

In [2428]:
eval_df = train_df_sampled.groupby('client_pin').tail(1)

In [2429]:
train_df_last = pd.merge(train_df_sampled, eval_df['session_id'], on='session_id', how='left', indicator=True)
train_df_last = train_df_last[train_df_last['_merge'] == 'left_only']

In [2430]:
train_df_sampled.shape, train_df_last.shape

((410809, 312), (349205, 313))

In [2432]:
lgbm.fit(train_df_sampled[cols], train_df_sampled['multi_class_target'], eval_set=(test_df_lin[cols], test_df_lin['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds
[5]	valid_0's multi_logloss: 1.60166	valid_0's f1_score: 0.396026
[10]	valid_0's multi_logloss: 1.50313	valid_0's f1_score: 0.403108
[15]	valid_0's multi_logloss: 1.46747	valid_0's f1_score: 0.404425
[20]	valid_0's multi_logloss: 1.45399	valid_0's f1_score: 0.404836
[25]	valid_0's multi_logloss: 1.44878	valid_0's f1_score: 0.403564
[30]	valid_0's multi_logloss: 1.44674	valid_0's f1_score: 0.40335
[35]	valid_0's multi_logloss: 1.4453	valid_0's f1_score: 0.403394
[40]	valid_0's multi_logloss: 1.44382	valid_0's f1_score: 0.404251
[45]	valid_0's multi_logloss: 1.44268	valid_0's f1_score: 0.403945
[50]	valid_0's multi_logloss: 1.44141	valid_0's f1_score: 0.404265
[55]	valid_0's multi_logloss: 1.44055	valid_0's f1_score: 0.403835
[60]	valid_0's multi_logloss: 1.4397	valid_0's f1_score: 0.404621
[65]	valid_0's multi_logloss: 1.43922	valid_0's f1_score: 0.404303
[70]	valid_0's multi_logloss: 1.43834	valid_0's f1_score: 0.403899
Early

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=210, n_jobs=64, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=12,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
rfc = RandomForestClassifier(n_estimators = 300, n_jobs = 64, max_depth = 5, criterion='entropy', min_impurity_decrease = 1e-6, random_state=1)

In [ ]:
rfc.fit()

In [2383]:
f1_score(test_df_lin['multi_class_target'], lgbm.predict(test_df_lin[new_atrs]), average='macro')

0.4058209441352839

In [2397]:
df_f = pd.DataFrame(zip(lgbm.feature_name_, lgbm.booster_.feature_importance(importance_type='gain')
), columns=['f', 'im']).sort_values(by='im', ascending=False)

In [2398]:
def get_new_features(df):
        for i in range(df.shape[0]):
            if df['im'].iloc[:i].sum()/df['im'].sum() > 0.999:
                break
        return df['f'].iloc[:i].values.tolist()

In [2399]:
new_atrs = get_new_features(df_f)

In [2437]:
submit_df_lin['prediction'] = lgbm.predict(submit_df_lin[cols])

In [2438]:
submit_df_lin[['client_pin', 'prediction']].to_csv('submission.csv', index=False)

In [1033]:
submit_df_lin['prediction'] = cv_clsf.predict(submit_df_lin[new_atrs])

In [1034]:
submit_df_lin[['client_pin','prediction']].to_csv('prediction_true.csv', index=False)

In [1029]:
# submit_df_lin

,device_screen_name_Add_New_Card_mean_prev_2,event_label_a7e467750a990887bfeb95f9384e146b_mean_prev_1,page_urlpath_full_46f90bdc46d30317d072d4c67d6cbfda_mean_prev_1,device_screen_name_Credit_Info_mean_prev_2,page_urlhost_online.alfabank.ru_mean_prev_1,event_label_5b14bf081cdfdbd3b37a3acb75bc553c_mean_prev_2,net_connection_tech_LTE_mean_prev_1,net_connection_tech_HSDPA_mean_prev_2,event_name_Select_one_of_last_transfer_mean_prev_2,prev_sum_mobile_recharge,...,top_target_parq_1_card2card_transfer,event_label_last_prev_1_card2card_transfer,timezone_prev_1_card2card_transfer,device_screen_name_last_prev_2_card2card_transfer,device_screen_name_last_prev_1_invest,top_target_parq_1_invest,event_label_last_prev_1_invest,timezone_prev_1_invest,device_screen_name_last_prev_2_invest,prediction
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.075713,0.063210,0.073499,0.080104,0.019348,0.019376,0.013211,1.853786e-02,0.040586,credit_info
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.075713,0.063210,0.073499,0.061309,0.021677,0.019376,0.013211,1.853786e-02,0.018698,statement
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.075713,0.063210,0.076061,0.063873,0.019348,0.019376,0.013211,2.335792e-02,0.018889,main_screen
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.075713,0.078689,0.076061,0.074743,0.024724,0.019376,0.035585,2.335792e-02,0.025449,statement
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.075713,0.063210,0.050505,0.070144,0.019348,0.019376,0.013211,7.070707e-02,0.016187,mobile_recharge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.075713,0.063210,0.076061,0.063873,0.021677,0.019376,0.013211,2.335792e-02,0.018889,statement
79264,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.075361,0.078689,0.076061,0.080104,0.019348,0.007256,0.035585,2.335792e-02,0.040586,main_screen
79265,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.075713,0.063210,0.076061,0.056121,0.051707,0.019376,0.013211,2.335792e-02,0.009957,statement
79266,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.075713,0.063225,0.076061,0.058790,0.012681,0.019376,0.032790,2.335792e-02,0.013508,statement


In [1030]:
submit_df.shape

(79268, 1680)

In [1880]:
train_df_lin['event_name_last_prev_1'] = train_df_lin['event_name_last_prev_1'].astype('category')

In [1881]:
test_df_lin['event_name_last_prev_1'] = test_df_lin['event_name_last_prev_1'].astype('category')

In [1406]:
cv_clsf = cb.CatBoostClassifier(objective='MultiClass', custom_metric=['F1'], verbose=True, eval_metric='TotalF1:average=Macro', random_seed=SEED, n_estimators=300)

In [1407]:
cv_clsf.fit(train_df_sampled[cols], train_df_sampled['multi_class_target'], eval_set=(test_df_lin[cols], test_df_lin['multi_class_target']), verbose=4, use_best_model=True, early_stopping_rounds=80)

Learning rate set to 0.192904
0:	learn: 0.2973776	test: 0.2519364	best: 0.2519364 (0)	total: 210ms	remaining: 1m 2s
4:	learn: 0.4286034	test: 0.3667490	best: 0.3667490 (4)	total: 897ms	remaining: 52.9s
8:	learn: 0.4455357	test: 0.3801044	best: 0.3801044 (8)	total: 1.77s	remaining: 57.2s
12:	learn: 0.4604595	test: 0.3900292	best: 0.3900292 (12)	total: 2.52s	remaining: 55.7s
16:	learn: 0.4657432	test: 0.3927874	best: 0.3928165 (15)	total: 3.28s	remaining: 54.6s
20:	learn: 0.4717303	test: 0.3965542	best: 0.3965542 (20)	total: 3.98s	remaining: 52.9s
24:	learn: 0.4747176	test: 0.3976446	best: 0.3976446 (24)	total: 4.68s	remaining: 51.5s
28:	learn: 0.4774744	test: 0.3987837	best: 0.3987837 (28)	total: 5.26s	remaining: 49.2s
32:	learn: 0.4793774	test: 0.3992544	best: 0.3992544 (32)	total: 5.9s	remaining: 47.7s
36:	learn: 0.4817215	test: 0.3995428	best: 0.3995428 (36)	total: 6.57s	remaining: 46.7s
40:	learn: 0.4831824	test: 0.4010462	best: 0.4011520 (39)	total: 7.26s	remaining: 45.9s
44:	learn

In [1098]:
%%time

f1_scores = []
for i in range(1100):
#     print(i)
    data = test_df_lin.sample(n=1000, random_state=i)
    f1_scores.append(f1_score(data['multi_class_target'], cv_clsf.predict(data[new_atrs]), average='macro'))

CPU times: user 2min 7s, sys: 7.71 s, total: 2min 15s
Wall time: 39.3 s


In [1099]:
np.mean(f1_scores)

0.4104406846396466

In [ ]:
f1_score(test_df_lin['multi_class_target'], lgbm.classes_[np.argmax(lgbm.predict_proba(test_df_lin[cols]) +  cv_clsf.predict_proba(test_df_lin[cols]), axis=1)])

### Применение нейронки

In [2418]:
from torch import nn
import torch

In [2419]:
class MLPclassifier(nn.Module):
    def __init__(self):
        super(MLPclassifier, self).__init__()
        self.linear_1 = nn.Linear(len(cols), 100)
#         self.batch_norm = nn.BatchNorm1d(100)
#         self.linear_2 = nn.Linear(300, 300)
#         self.linear_3 = nn.Linear(300, 150)
        self.linear_out = nn.Linear(100, len(df_lin['multi_class_target'].unique()))
    def forward(self, x):
        x = nn.functional.relu(self.linear_1(x))
                
#         x = self.batch_norm(x)
#         x = nn.functional.relu(self.linear_2(x))
#         x = nn.functional.relu(self.linear_3(x))

        output = self.linear_out(x)
        return output

In [2420]:
from sklearn.preprocessing import MinMaxScaler

In [2421]:
X_train, X_test = train_df_sampled[cols], test_df_lin[cols]
y_train, y_test = train_df_sampled['multi_class_target'], test_df_lin['multi_class_target']

In [2422]:
lb_target = LabelEncoder()
lb_target.fit(train_df_lin['multi_class_target'])

y_train = lb_target.transform(y_train)
y_test = lb_target.transform(y_test)

In [2423]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ValueError: could not convert string to float: 'statement'

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ml_class = MLPclassifier()
ml_class.train()

In [ ]:
learning_rate = 1e-5
optimizer = torch.optim.Adam(ml_class.parameters(), lr=learning_rate, weight_decay=0.2)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.RMSprop(ml_class.parameters(), lr=learning_rate, weight_decay=0.2)

In [ ]:
torch_tensor_test = torch.from_numpy(x_b.astype(np.float32))    

In [ ]:
# learning_rate = 1e-3

# optimizer = torch.optim.Adam(ml_class.parameters(), lr=learning_rate, weight_decay=0)
# # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 20)

In [ ]:
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [ ]:
torch.manual_seed(SEED)
ml_class.apply(weight_reset)

In [ ]:
steps = 20

In [ ]:
learning_rate = 0.1
weight_decay = 0.005
momentum = 0.9

optimizer = torch.optim.SGD(ml_class.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

loss_fn = torch.nn.CrossEntropyLoss()

In [2424]:
torch_tensor_test = torch.from_numpy(X_test.astype(np.float32))

for t in range(3000):
    np.random.seed(t)
    
    ind_train = np.random.choice(X_train.shape[0], int(3e4), replace=False)
    
    x, y = X_train[ind_train], y_train[ind_train]
    
    torch_tensor = torch.from_numpy(x.astype(np.float32))
    torch_tensor_target = torch.from_numpy(y)

    y_pred = ml_class(torch_tensor)
    loss = loss_fn(y_pred, torch_tensor_target)
   
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    scheduler.step()
    
    if t% steps == 0:
        print('На трейне:', loss)
        print('На трейне f1:', f1_score(y, y_pred.argmax(dim=-1).cpu().detach().numpy(), average='macro'))
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
        y_pred = ml_class(torch_tensor_test).argmax(dim=-1).cpu().detach().numpy()
        print(f1_score(y_test, y_pred, average='macro'))

ValueError: could not convert string to float: 'main_screen'

In [2855]:
X_test

array([[0.        , 0.        , 0.00382777, ..., 0.03536412, 0.03136918,
        0.00547327],
       [0.        , 0.        , 0.01056873, ..., 0.01496774, 0.03136918,
        0.00561277],
       [0.        , 0.        , 0.00973864, ..., 0.04549195, 0.04546028,
        0.02091962],
       ...,
       [0.09097706, 0.        , 0.26422826, ..., 0.02556818, 0.03136918,
        0.02091962],
       [0.00328901, 0.        , 0.00050704, ..., 0.04549195, 0.03136918,
        0.04621706],
       [0.01014549, 0.00200401, 0.00066603, ..., 0.01496774, 0.03136918,
        0.02091962]])

### Применение catboost

In [112]:
cv_clsf = cb.CatBoostClassifier(objective='MultiClass', custom_metric=['F1'], verbose=True, eval_metric='TotalF1:average=Macro', random_seed=SEED)

In [114]:
cv_clsf.fit(train_df_less[atrs], train_df_less['multi_class_target'], eval_set=(test_df[atrs], test_df['multi_class_target']), verbose=4, cat_features=cat_atrs, use_best_model=True, early_stopping_rounds=50)

KeyboardInterrupt: 

#### Отброс фичей c ненулевой важностью

In [879]:
featue_importance = pd.DataFrame(zip(cv_clsf.feature_names_, cv_clsf.feature_importances_)).sort_values(by=1, ascending=False)
featue_importance[featue_importance[1] != 0]

,0,1
127,top_target,18.184095
58,target_1,16.110004
60,target_2,5.693938
52,prev_sum_own_transfer,4.479359
128,top_target_2,3.770082
63,target_3,2.780499
44,bigger_credit_info_card2card_transfer,2.494685
48,prev_sum_credit_info,2.259087
46,prev_sum_card_recharge,2.221973
43,bigger_card_recharge_card2card_transfer,1.856740


In [880]:
featue_importance = featue_importance[featue_importance[1] != 0]

In [881]:
new_features = featue_importance[0].values.tolist()

In [882]:
cv_clsf.fit(train_df_less[new_features], train_df_less['multi_class_target'], eval_set=(test_df[new_features], test_df['multi_class_target']), verbose=4, cat_features=[atr for atr in cat_atrs if atr in new_features], use_best_model=True, early_stopping_rounds=100)

Learning rate set to 0.122333
0:	learn: 0.3859987	test: 0.3341666	best: 0.3341666 (0)	total: 3.21s	remaining: 53m 27s
4:	learn: 0.4290241	test: 0.3619413	best: 0.3619413 (4)	total: 19.7s	remaining: 1h 5m 18s
8:	learn: 0.4363532	test: 0.3711523	best: 0.3711523 (8)	total: 34.9s	remaining: 1h 3m 59s
12:	learn: 0.4503509	test: 0.3809214	best: 0.3809214 (12)	total: 52s	remaining: 1h 5m 48s
16:	learn: 0.4562094	test: 0.3856448	best: 0.3856448 (16)	total: 1m 9s	remaining: 1h 7m 2s
20:	learn: 0.4623137	test: 0.3898220	best: 0.3898220 (20)	total: 1m 25s	remaining: 1h 6m 38s
24:	learn: 0.4709211	test: 0.3946393	best: 0.3946393 (24)	total: 1m 42s	remaining: 1h 6m 41s
28:	learn: 0.4763981	test: 0.3968804	best: 0.3968804 (28)	total: 2m 1s	remaining: 1h 8m 4s
32:	learn: 0.4797402	test: 0.3979614	best: 0.3979614 (32)	total: 2m 19s	remaining: 1h 8m 2s
36:	learn: 0.4817473	test: 0.3985023	best: 0.3988719 (35)	total: 2m 36s	remaining: 1h 7m 41s
40:	learn: 0.4838562	test: 0.4005579	best: 0.4005579 (40)	t

### Более полная оценка точности на тестовой выборке

In [156]:
test_df['prediction'] = cv_clsf.predict(test_df[new_features])

NameError: name 'new_features' is not defined

In [157]:
from sklearn.metrics import classification_report

In [158]:
pd.DataFrame(classification_report(test_df['multi_class_target'], test_df['prediction'], output_dict=True)).transpose()

KeyError: 'prediction'

In [821]:
test_df['multi_class_target']

28717                 statement
47637               main_screen
71964      phone_money_transfer
104513              main_screen
111031              main_screen
                   ...         
5144529               statement
5144536             main_screen
5144543             main_screen
5144547            own_transfer
5144562         mobile_recharge
Name: multi_class_target, Length: 77631, dtype: object

In [822]:
from sklearn.metrics import confusion_matrix

In [807]:
labels=test_df['multi_class_target'].unique().tolist()

In [808]:
pd.DataFrame(confusion_matrix(test_df['multi_class_target'], test_df['prediction'], labels=labels), columns=labels, index=labels)

,statement,main_screen,phone_money_transfer,own_transfer,invest,credit_info,mobile_recharge,card2card_transfer,chat,card_recharge
statement,5769,2207,751,638,49,1029,620,426,529,653
main_screen,4125,17713,1416,1314,116,3567,1255,859,1171,2041
phone_money_transfer,701,690,1351,361,25,169,260,230,110,138
own_transfer,272,299,200,1819,36,254,143,103,109,95
invest,88,136,39,78,258,58,18,21,59,49
credit_info,474,1024,101,312,20,7111,137,150,496,575
mobile_recharge,553,486,301,255,10,176,1164,188,65,95
card2card_transfer,317,413,189,169,10,190,178,809,89,113
chat,529,703,141,206,25,606,103,88,1281,252
card_recharge,208,352,44,65,7,540,33,50,133,1678


### Получение сабмит файла

In [823]:
submit_df.head(n=1)

,bigger_mobile_recharge_statement,bigger_mobile_recharge_phone_money_transfer,bigger_mobile_recharge_chat,bigger_mobile_recharge_invest,bigger_mobile_recharge_main_screen,bigger_mobile_recharge_own_transfer,bigger_mobile_recharge_card_recharge,bigger_mobile_recharge_credit_info,bigger_mobile_recharge_card2card_transfer,bigger_statement_phone_money_transfer,...,page_urlhost_last,page_urlpath_full_first,page_urlpath_full_last,net_connection_type_first,net_connection_type_last,net_connection_tech_first,net_connection_tech_last,session_id_size,session_diff_time,prediction
4354786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,nan,nan,nan,mobile,mobile,LTE,LTE,9.0,113.031,credit_info


In [159]:
submit_df['prediction'] = cv_clsf.predict(submit_df[atrs])

In [160]:
submit_df[['client_pin', 'prediction']].to_csv('submit.csv', index=False)

In [112]:
atrs

['bigger_mobile_recharge_statement',
 'bigger_mobile_recharge_phone_money_transfer',
 'bigger_mobile_recharge_chat',
 'bigger_mobile_recharge_invest',
 'bigger_mobile_recharge_main_screen',
 'bigger_mobile_recharge_own_transfer',
 'bigger_mobile_recharge_card_recharge',
 'bigger_mobile_recharge_credit_info',
 'bigger_mobile_recharge_card2card_transfer',
 'bigger_statement_phone_money_transfer',
 'bigger_statement_chat',
 'bigger_statement_invest',
 'bigger_statement_main_screen',
 'bigger_statement_own_transfer',
 'bigger_statement_card_recharge',
 'bigger_statement_credit_info',
 'bigger_statement_card2card_transfer',
 'bigger_phone_money_transfer_chat',
 'bigger_phone_money_transfer_invest',
 'bigger_phone_money_transfer_main_screen',
 'bigger_phone_money_transfer_own_transfer',
 'bigger_phone_money_transfer_card_recharge',
 'bigger_phone_money_transfer_credit_info',
 'bigger_phone_money_transfer_card2card_transfer',
 'bigger_chat_invest',
 'bigger_chat_main_screen',
 'bigger_chat_ow

In [73]:
new_atr = [atr for atr in atrs if 'sum'  not in atr]

In [74]:
lgbm.fit(train_df_less[new_atr], train_df_less['multi_class_target'], eval_set=(test_df[new_atr], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds
[5]	valid_0's multi_logloss: 1.59652	valid_0's f1_score: 0.3992
[10]	valid_0's multi_logloss: 1.49918	valid_0's f1_score: 0.404926
[15]	valid_0's multi_logloss: 1.46483	valid_0's f1_score: 0.405171
[20]	valid_0's multi_logloss: 1.45161	valid_0's f1_score: 0.40493
[25]	valid_0's multi_logloss: 1.44616	valid_0's f1_score: 0.404693
[30]	valid_0's multi_logloss: 1.44368	valid_0's f1_score: 0.405082
[35]	valid_0's multi_logloss: 1.44246	valid_0's f1_score: 0.404335
[40]	valid_0's multi_logloss: 1.442	valid_0's f1_score: 0.40393
[45]	valid_0's multi_logloss: 1.44147	valid_0's f1_score: 0.40429
[50]	valid_0's multi_logloss: 1.4414	valid_0's f1_score: 0.404732
[55]	valid_0's multi_logloss: 1.44122	valid_0's f1_score: 0.405071
[60]	valid_0's multi_logloss: 1.44121	valid_0's f1_score: 0.405226
[65]	valid_0's multi_logloss: 1.44092	valid_0's f1_score: 0.405185
[70]	valid_0's multi_logloss: 1.44085	valid_0's f1_score: 0.40517
[75]	valid_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=200, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=0.9,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [56]:
prev_columns = [col for col in train_df_less.columns if 'prev_sum' in col]

In [58]:
for col in prev_columns:
    train_df_less['mean' + col.split('prev_sum')[1]] = train_df_less[col]/train_df_less['sessions_number']
    test_df['mean' + col.split('prev_sum')[1]] = test_df[col]/test_df['sessions_number']

In [144]:
# lgbm.fit(train_df_lgb[lgbm_atrs], train_df_lgb['multi_class_target'], eval_set=(test_df_lgb[lgbm_atrs], test_df_lgb['multi_class_target']), eval_metric=evalerror, verbose=20, early_stopping_rounds=100)

In [149]:
len(atrs)

165

In [168]:
df_f = pd.DataFrame(zip(lgbm.feature_name_, lgbm.feature_importances_)).sort_values(by=1, ascending=False)

In [169]:
df_f.iloc[:-50, 1].sum()/df_f[1].sum()

0.9848563968668407

In [170]:
new_features = df_f.iloc[:-40, 0].values

In [171]:
lgbm_new = LGBMClassifier(objective='multiclass', reg_alpha=0.9, max_depth=5, min_split_gain = 0.7, subsample=0.7, n_estimators=200, n_jobs=60, random_state=SEED)

In [173]:
lgbm_new.fit(train_df_less[new_features], train_df_less['multi_class_target'], eval_set=(test_df[new_features], test_df['multi_class_target']), eval_metric=evalerror, verbose=5, early_stopping_rounds=50)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 100 rounds
[5]	valid_0's multi_logloss: 1.60118	valid_0's f1_score: 0.399173
[10]	valid_0's multi_logloss: 1.4985	valid_0's f1_score: 0.408484
[15]	valid_0's multi_logloss: 1.45821	valid_0's f1_score: 0.410954
[20]	valid_0's multi_logloss: 1.43967	valid_0's f1_score: 0.410968
[25]	valid_0's multi_logloss: 1.43152	valid_0's f1_score: 0.412343
[30]	valid_0's multi_logloss: 1.42751	valid_0's f1_score: 0.412323
[35]	valid_0's multi_logloss: 1.42555	valid_0's f1_score: 0.412411
[40]	valid_0's multi_logloss: 1.42426	valid_0's f1_score: 0.412506
[45]	valid_0's multi_logloss: 1.42366	valid_0's f1_score: 0.412644
[50]	valid_0's multi_logloss: 1.42281	valid_0's f1_score: 0.41289
[55]	valid_0's multi_logloss: 1.42246	valid_0's f1_score: 0.413762
[60]	valid_0's multi_logloss: 1.42241	valid_0's f1_score: 0.413797
[65]	va

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.7,
               n_estimators=200, n_jobs=60, num_leaves=31,
               objective='multiclass', random_state=1, reg_alpha=0.9,
               reg_lambda=0.0, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=0)

In [176]:
f1_score(test_df['multi_class_target'], cv_clsf.classes_[np.argmax(cv_clsf.predict_proba(test_df[atrs]) + lgbm.predict_proba(test_df[atrs]) + lgbm_new.predict_proba(test_df[new_features]), 1)], average='macro')

0.414607849230933

In [178]:
submit_df['prediction'] = cv_clsf.classes_[np.argmax(cv_clsf.predict_proba(submit_df[atrs]) + lgbm.predict_proba(submit_df[atrs]) + lgbm_new.predict_proba(submit_df[new_features]), 1)]

In [179]:
submit_df[['client_pin', 'prediction']].to_csv('blending.csv', index=False)

### Применение xgboost

In [344]:
from xgboost import XGBClassifier

In [361]:
import xgboost

xgboost.__version__

'0.90'

In [362]:
train_df['multi_class_target'].unique()

array(['phone_money_transfer', 'main_screen', 'own_transfer',
       'credit_info', 'chat', 'statement', 'card_recharge',
       'card2card_transfer', 'mobile_recharge', 'invest'], dtype=object)

In [ ]:
train_df_lgb['multi_class_target'].uni

In [357]:
xgb = XGBClassifier(objective='multi:softmax', num_class=10)

In [368]:
train_df_lgb['multi_class_target'].unique()

array(['invest', 'mobile_recharge', 'card2card_transfer', 'card_recharge',
       'statement', 'chat', 'credit_info', 'own_transfer', 'main_screen',
       'phone_money_transfer'], dtype=object)

In [370]:
lb_target = LabelEncoder()
lb_target.fit(train_df_lgb['multi_class_target'])

LabelEncoder()

In [373]:
train_df_lgb['multi_class_target'].unique()

array(['invest', 'mobile_recharge', 'card2card_transfer', 'card_recharge',
       'statement', 'chat', 'credit_info', 'own_transfer', 'main_screen',
       'phone_money_transfer'], dtype=object)

In [375]:
test_df_lgb['multi_class_target'].unique()

array(['statement', 'main_screen', 'phone_money_transfer', 'own_transfer',
       'invest', 'credit_info', 'mobile_recharge', 'card2card_transfer',
       'chat', 'card_recharge'], dtype=object)

In [382]:
xgb.fit(train_df_lgb[lgbm_atrs].values, lb.transform(train_df_lgb['multi_class_target'].values), eval_set=(test_df_lgb[lgbm_atrs].values, lb.transform(test_df_lgb['multi_class_target'].values)), verbose=5, early_stopping_rounds=20)

ValueError: y contains previously unseen labels: 3836.15